In [4]:
import pandas as pd
import numpy as np
import time
import psycopg2
import json
import sys
import os


#### Test parametrically changing the number of columns in pandas DataFrames

In [5]:
##### basic repetition of making a Dataframe

In [6]:
basic_test_data_frame = pd.DataFrame( data={"p1": np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [pd.Timestamp.now() ] ) )
basic_test_data_frame

,p1,p2
2019-10-15 09:23:19.242829,NaN,NaN


In [7]:
#### Now try making a dataframe parametrically

In [8]:
db_data_array = {}

In [9]:
num_of_df_cols = 5

In [10]:
for col_num in range( num_of_df_cols ):
    curr_col_name = "col"+str( col_num )
    db_data_array[ curr_col_name ] = col_num

In [11]:
db_data_array

{'col0': 0, 'col1': 1, 'col2': 2, 'col3': 3, 'col4': 4}

In [12]:
# now make dataframe! 
random_pd_df = pd.DataFrame( data=db_data_array, index=pd.DatetimeIndex( [pd.Timestamp.now() ] ))
random_pd_df

,col0,col1,col2,col3,col4
2019-10-15 09:23:20.379209,0,1,2,3,4


#### test resampling : 

In [13]:
# make a data frame … 
random_pd_df_2 = pd.DataFrame( data=db_data_array, index=pd.DatetimeIndex( [pd.Timestamp( 2019, 10, 15, 9) ] ))
random_pd_df_2

,col0,col1,col2,col3,col4
2019-10-15 09:00:00,0,1,2,3,4


In [14]:
random_pd_df_3 = random_pd_df_2*2

In [24]:
random_pd_df_3.index = pd.DatetimeIndex( [pd.Timestamp(2019, 10, 15, 10) ] )
random_pd_df_3

,col0,col1,col2,col3,col4
2019-10-15 10:00:00,0,2,4,6,8


In [53]:
random_pd_df_4 = random_pd_df_2.append( random_pd_df_3 )
random_pd_df_4

,col0,col1,col2,col3,col4
2019-10-15 09:00:00,0,1,2,3,4
2019-10-15 10:00:00,0,2,4,6,8


In [56]:
# basic resample
# sum resample - values fall into a bin ( good for downsampling )
random_pd_df_4__resampled = random_pd_df_4.resample( '10T' ).sum()
random_pd_df_4__resampled

,col0,col1,col2,col3,col4
2019-10-15 09:00:00,0,1,2,3,4
2019-10-15 09:10:00,0,0,0,0,0
2019-10-15 09:20:00,0,0,0,0,0
2019-10-15 09:30:00,0,0,0,0,0
2019-10-15 09:40:00,0,0,0,0,0
2019-10-15 09:50:00,0,0,0,0,0
2019-10-15 10:00:00,0,2,4,6,8


In [55]:
# pad resample  - essentially use the previous value to fill in the blanks 
random_pd_df_4__resampled = random_pd_df_4.resample( '10T' ).pad()
random_pd_df_4__resampled

,col0,col1,col2,col3,col4
2019-10-15 09:00:00,0,1,2,3,4
2019-10-15 09:10:00,0,1,2,3,4
2019-10-15 09:20:00,0,1,2,3,4
2019-10-15 09:30:00,0,1,2,3,4
2019-10-15 09:40:00,0,1,2,3,4
2019-10-15 09:50:00,0,1,2,3,4
2019-10-15 10:00:00,0,2,4,6,8


In [46]:
# bfill resample ( average ? )
random_pd_df_4__resampled = random_pd_df_4.resample( '10T' ).bfill()
random_pd_df_4__resampled

,col0,col1,col2,col3,col4
2019-10-15 09:00:00,0,1,2,3,4
2019-10-15 09:10:00,0,2,4,6,8
2019-10-15 09:20:00,0,2,4,6,8
2019-10-15 09:30:00,0,2,4,6,8
2019-10-15 09:40:00,0,2,4,6,8
2019-10-15 09:50:00,0,2,4,6,8
2019-10-15 10:00:00,0,2,4,6,8


In [48]:
random_pd_df_4__resampled.shape, random_pd_df_4__resampled.size

((7, 5), 35)